# MENTORÍA 06
# **Cambio climático y ML: cómo mitigar las emisiones de CO2 mediante la reducción del consumo energético en construcciones edilicias**

**TP N°2: Análisis exploratorio y curación**

Fecha de entrega: **29 de julio**

## Consignas generales

En este segundo TP deberemos asegurarnos de que nuestro dataset esté listo para introducirlo a un modelo. 
Al igual que en el TP anterior, les propongo la elaboración de un informe donde se aclare o se detalle las decisiones tomadas.
Podemos tener más de un set de datos (o escenarios), de forma tal de poder analizar combinaciones diferentes de features que creamos puedan tener incidencia en nuestras predicciones. Les propongo contar con al menos dos escenarios (pueden evaluar más de dos, por supuesto):

    - Escenario base: datos sin escalar y con todas las features.
    - Escenario 2: datos escalados y con las mejoras que hayan realizado.
    
De esta manera, podremos comparar qué tan buenas fueron nuestras elecciones, al contrastar las métricas que obtendremos con el peor de los modelos que podríamos haber usado.

## Pasos a seguir para evitar el train-test data leaking

1- Renombrar columnas o transformar unidades de medición.

2- Chequear datos repetidos.

3- Chequear "missingness" o valores faltantes:
 
Verificar que las columnas numéricas correspondan a un tipo de dato numérico. Muchas veces, los valores faltantes se llenan con algún string como "NA", '.', o "-". En ese caso, un dato numérico (float o int) tendrá el tipo object.
    
        1- df.info() --> chequeo tipo de datos
        2- df.coldeinteres.unique() --> si veo una columna misteriosa, analizo los valores que contiene
        3- df.describe --> Si con un describe (o un value_counts) identifican valores == 0 en features que no deberían tener 0, entonces, ahí hay otro dato falante incorporado como 0. 
 
Si encuentran algo así, deben reemplazar esos valores con "missing value" o NaN:

        1- df = pd.read_csv('data.csv', na_values='-')
        2- df.replace('-', np.nan)
        2- df.coldeinteres[df.coldeinteres == 0] = np.nan

Si deciden eliminar alguna columna por presentar muchos valores faltantes, pueden hacerlo en esta etapa. 

Pero, si deciden imputar, deberán hacerlo más adelante.
    
----------------------------------------------------------------------------------------------------------------------------------

Los puntos anteriores los pueden realizar con la totalidad del dataset. Sin embargo, previo a trabajar con las variables numéricas o cuantitativas que requieran escalado, chequeo de outliers, o evaluación de valores faltantes para imputar, debemos **DIVIDIR EL DATA SET EN TRAIN Y TEST**. De lo contrario, estaríamos comentiendo lo que se conoce como **contaminación train-test (un tipo de data leaking).** 

-----------------------------------------------------------------------------------------------------------------------------------

4- Separar la variable target y dividir el set de datos en train y test.

        X = df.drop(['target'],axis=1)
        y = df['target']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)        
        
5- Luego de identificar las variables categóricas o cualitativas, evaluar su distribución, reasignarlas (o no), encodearlas por separado. Pueden usar getdummies, OneHotEncoding o OrdinalEncoding, dependiendo el objetivo.
Durante el proceso, recordar dropear una de las categorías para evitar la multicolinealidad.
    
        X_train = pd.get_dummies(X_train, columns=["catcol1", "catcol2"], drop_first=True)
        X_test = pd.get_dummies(X_test, columns=["catcol1", "catcol2"], drop_first=True)
        
6- Escalar variables numéricas con el método .fit_transform en el set de entrenamiento, pero sólo con el método .transform en el set de testeo. Esto se hace para utilizar la media y el desvío calculados (si usamos StandardScaler, por ejemplo) únicamente con los datos de entrenamiento. Recuerden que los datos de test son datos que simulan provenir de la realidad, posterior al entrenamiento del modelo, y su función radica en testear que tan bien generaliza mi modelo con datos nuevos (ya que para eso estamos haciendo y entrenando nuestros modelos: para que predigan lo mejor posible sobre nuevos valores de features que nunca vio). Ejemplos:

        scaler = StandardScaler()
        scaler.fit_transform(X_train)
        scaler.transform(X_test)
        
        scaler = MinMaxScaler()
        num_cols = ["numcol1","numcol2","numcol3"]
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_test[num_cols] = scaler.transform(X_test[num_cols])

7- El proceso de imputar es similar al de escalado:

        # Imputación
        my_imputer = SimpleImputer()
        imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
        imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

        # Como la imputación elimina los nombres de las columnas, hay que agregarlos de nuevo
        imputed_X_train.columns = X_train.columns
        imputed_X_test.columns = X_test.columns
        
-------------------------------------------------------------------------------------------------------------------------------------

En cuanto al análisis de los outliers, va a depender si los necesitamos para corregir valores atípicos (temperaturas de 1000°C, por ejemplo) o si vamos a depender del análisis de la distribución de los datos y, por ende, de la mediana. En el primer caso, podemos usar todo el dataset. Pero en el segundo, deberíamos trabajar el set de train y le de test por separado. 


Fuentes:

https://jahazielponce.com/kaggle-30-dias-ml-dias-12-14/

https://towardsdatascience.com/the-dreaded-antagonist-data-leakage-in-machine-learning-5f08679852cc